In [1]:
import matplotlib
matplotlib.use('Qt5Agg')

In [2]:
import numpy as np
import mne
from mne.datasets import sample

In [3]:
data_path = sample.data_path()

Wczytujemy dane

In [5]:
raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
event_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
tmin, tmax = -0.1, 0.3

raw = mne.io.read_raw_fif(raw_fname, preload=True)

raw.filter(0.1, 60, fir_design='firwin')
events = mne.read_events(event_fname)

Opening raw data file C:\Users\marcin\mne_data\MNE-sample-data\MEG\sample\sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Current compensation grade : 0
Reading 0 ... 41699  =      0.000 ...   277.709 secs...
Setting up band-pass filter from 0.1 - 60 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 15.0 Hz
Filter length of 4955 samples (33.000 sec) selected


In [6]:
raw.plot()

<Figure size 3840x2052 with 5 Axes>

Usunięcie komponentów ocznych za pomocą regresji.

In [7]:
raw2 = raw.copy()

In [8]:
eog_chan = mne.pick_channels(raw2.ch_names, ['EOG 061'])
EOG = raw2.get_data()[eog_chan, :]

In [9]:
picks = mne.pick_types(raw2.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
EEG = raw2.get_data()[picks, :]

In [10]:
b = np.linalg.inv(EOG @ EOG.T) @ EOG @ EEG.T
EEG_corrected = (EEG.T - EOG.T @ b).T

In [11]:
raw2._data[picks, :] = EEG_corrected

In [12]:
raw2.plot()

<Figure size 3840x2052 with 5 Axes>

Usunięcie ocznych artefaktów za pomocą ICA

In [13]:
from mne.preprocessing import ICA

In [14]:
raw.pick_types(meg=False, eeg=True, eog=True, exclude='bads')

<Raw  |  sample_audvis_filt-0-40_raw.fif, n_channels x n_times : 60 x 41700 (277.7 sec), ~22.1 MB, data loaded>

In [15]:
ica_data = ICA(method='fastica', random_state=0, max_iter=500)
ica_data.fit(raw)

Fitting ICA to data using 59 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 59
Fitting ICA took 2.0s.


<ICA  |  raw data decomposition, fit (fastica): 41700 samples, 59 components, channels used: "eeg">

In [16]:
ica_data.plot_components(range(20), inst=raw)

<Figure size 750x696 with 20 Axes>

In [17]:
ica_data.exclude = [0]

In [18]:
ica_data.apply(raw)

Transforming to ICA space (59 components)
Zeroing out 1 ICA components


<Raw  |  sample_audvis_filt-0-40_raw.fif, n_channels x n_times : 60 x 41700 (277.7 sec), ~22.1 MB, data loaded>

In [19]:
raw.plot()

<Figure size 3840x2048 with 5 Axes>

Wybieramy dane, które będą nas interesować.

In [20]:
events

array([[ 6994,     0,     2],
       [ 7086,     0,     3],
       [ 7192,     0,     1],
       [ 7304,     0,     4],
       [ 7413,     0,     2],
       [ 7506,     0,     3],
       [ 7612,     0,     1],
       [ 7709,     0,     4],
       [ 7810,     0,     2],
       [ 7916,     0,     3],
       [ 8025,     0,     1],
       [ 8129,     0,     4],
       [ 8233,     0,     2],
       [ 8331,     0,     3],
       [ 8428,     0,     1],
       [ 8522,     0,     5],
       [ 8633,     0,     2],
       [ 8662,     0,    32],
       [ 8739,     0,     3],
       [ 8857,     0,     1],
       [ 8962,     0,     4],
       [ 9052,     0,     2],
       [ 9144,     0,     3],
       [ 9251,     0,     1],
       [ 9365,     0,     4],
       [ 9477,     0,     2],
       [ 9581,     0,     3],
       [ 9677,     0,     1],
       [ 9782,     0,     4],
       [ 9890,     0,     2],
       [ 9981,     0,     3],
       [10101,     0,     1],
       [10220,     0,     4],
       [10

In [21]:
event_id = {'left/auditory': 1, 'right/auditory': 2,
            'left/visual': 3, 'right/visual': 4}

In [22]:
picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False, picks=picks, baseline=None, preload=True, verbose=False)

In [23]:
x = epochs.average()
x.plot_joint(times=[0.1], ts_args=dict(time_unit='s'), topomap_args=dict(time_unit='s'));

In [24]:
epochs['auditory','left'].average().plot_joint(times=[0.1], ts_args=dict(time_unit='s'), topomap_args=dict(time_unit='s'));

In [25]:
epochs['right'].average().plot_joint(times=[0.1], ts_args=dict(time_unit='s'), topomap_args=dict(time_unit='s'));